# Creating a new system
This notebook shows how we can generate the atomic coordinates for a simple crystalline or liquid Ar system, which can be used to start MD simulations using openMM

In [ ]:
import time
import random

import numpy as np
import MDAnalysis as md
import nglview as ng

t = int( time.time() * 1000.0 )
# random.seed(123456) # <- use this for reproducibility
random.seed(int(time.time()))

We define some variables that may come in handy to improve the readability of the code later.

In [ ]:
X = 0
Y = 1
Z = 2

# Cartesian vectors
xVec = np.array([[1.0 , 0.0 , 0.0]])
yVec = np.array([[0.0 , 1.0 , 0.0]])
zVec = np.array([[0.0 , 0.0 , 1.0]])

We then define a function that will write the atomic coordinates in PDB format, which has some very strict requirements

In [ ]:
def writeCoordinatesPDB(filename):
    n_residues = numberOfAtoms

    # create resindex list
    resindices = np.repeat(range(n_residues), 1)
    assert len(resindices) == numberOfAtoms

    # all water molecules belong to 1 segment
    segindices = [0] * n_residues

    u = md.Universe.empty(numberOfAtoms,
                             n_residues=n_residues,
                             atom_resindex=resindices,
                             residue_segindex=segindices,
                             trajectory=True) # necessary for adding coordinates

    # cell
    u.dimensions = [systemSize[X],systemSize[Y],systemSize[Z], 90, 90, 90]

    # names
    u.add_TopologyAttr('record_types', ['HETATM']*numberOfAtoms)
    u.add_TopologyAttr('name', ['Ar']*numberOfAtoms)
    u.add_TopologyAttr('element', ['Ar']*numberOfAtoms)

    # positions
    u.atoms.positions = positions

    # resname
    u.add_TopologyAttr('resname', ['UNK']*n_residues)

    # res IDs
    u.add_TopologyAttr('resid', list(range(1, n_residues+1)))

    u.add_TopologyAttr('segid', [' '])
    u.add_TopologyAttr('occupancies', [1.0]*numberOfAtoms)
    u.add_TopologyAttr('tempfactors', [1.0]*numberOfAtoms)
    u.add_TopologyAttr('chainIDs', ['X']*n_residues)
    u.add_TopologyAttr('altLocs', [" "]*n_residues)
    u.add_TopologyAttr('icodes', [" "]*n_residues)
    
    system = u.select_atoms("all")
    system.write(filename)

The following function creates a random distribution of 1000 atoms in 3D.
The first line defined the size of the simulation cell.
The function returns the number of atoms added, the system size and the atoms' positions

In [ ]:
def createLiquid():
    systemSize = np.array([40. , 40. , 40.])

    numberOfAtoms = 1000
    pos = np.zeros((numberOfAtoms,3))
    
    for i in range(numberOfAtoms):
        pos[i,:] = [random.random(),random.random(),random.random()] * systemSize
        
    return numberOfAtoms, systemSize , pos

The following function creates an FCC crystal or Argon.
The first line defines the size of the unit cell, and then we define the positions of the atoms in the unit cell using fractional coordinates. Finally we set the number of repeats of the unit cell that we want to create in each Cartesian direction.
The function returns the number of atoms added, the system size and the atoms' positions

In [ ]:
def createCrystalFCC():
    unitCell = np.array([5.51 , 5.51 , 5.51])

    nUnit = 4
    refPos = np.array([[0.0,0.0,0.0],
                       [0.0,0.5,0.5],
                       [0.5,0.0,0.5],
                       [0.5,0.5,0.0]])

    nrepl = np.array([6,6,6])

    systemSize = unitCell * nrepl
    numberOfAtoms = nUnit*np.product(nrepl)
    pos = np.zeros((numberOfAtoms,3))
    n = 0
    for ix in range(0,nrepl[X]):
        for iy in range(0,nrepl[Y]):
            for iz in range(0,nrepl[Z]):
                for i in range(0,nUnit):
                    pos[n,:] = (refPos[i,:] + ix*xVec + iy*yVec + iz*zVec) * unitCell
                    n += 1

    return numberOfAtoms, systemSize , pos

We finally write out the coordinates

In [ ]:
numberOfAtoms , systemSize, positions = createLiquid()
# numberOfAtoms , systemSize, positions = createCrystalFCC()

pdbFile = "coord.pdb"
writeCoordinatesPDB(pdbFile)

We can make sure that everything worked by loading up the file we have just written and visualise it using nglview.

In [ ]:
sys = md.Universe(pdbFile)
view = ng.show_mdanalysis(sys, gui=True)
view.center()
view.representations = [
    {"type": "spacefill", "params": {"sele": "all"}},
    {"type": "unitcell", "params": {"sele": "all"}}
]
view.camera = 'orthographic'
view
